# Array-based recast of DECSKS p2: <font color = "red">DECSKS-v1.2</font> $\Rightarrow$ <font color = "green">DECSKS-v2.0</font>

#### For easy identification: changes to the code are titled in <font color = "green">green</font> for DECSKS-v2.0 and in <font color = "red">red</font> for the old DECSKS-v1.2 code that is being analyzed here to discern the appropriate changes.

At this point we have created c (c.shape = (N, v.N)) and d (d.shape = (N, x.N, v.N)). Each column c[:,j] corresponds to the 1D advection problem for all $x$ MCs at a velocity $v_j$ on the mesh. Each 1D array d[dn,:,j] are the corresponding derivative coefficients for all $x$ at velocity $v_j$ for a derivative number $dn$ ($\partial_x^{dn}$).

For convenient referencing, the flux code is repeated here:

####<font color = "red">DECSKS-v1.2: lib.convect.flux excerpt</font> 

In [ ]:
    for q in range(1,sim_params['N']):
        Uf += c[q]*d[:,q] # Flux = Uf + H.O.C. [High Order Corrections]

Above, we are solving one 1D problem at a time, so c.shape = (N,), and d.shape = (x.N or v.N, N)

Uf is updated according to an incremental update += above. This accomplishes the aforementioned HOC sum:


$$[Uf]_{i,j} = \sum\limits_{q = 0}^{N-1} c_{q,i}d^{(q)}_{i,j}$$

at each $i,j$.

We require pair-wise multiplication in the first dimension of each arrow and summing over them. This can be done with an inner product included as a single addition assignment instead of looping through and incrementing per +=.

The above computation 

                for q in range(1, sim_params['N']):
                    Uf += c[q]*d[:,q] 
    
is now equivalent to in the 2D repurposing as:

                Uf = np.zeros_like(f[n,:,:])
                
                for q in range(sim_params['N']):
                    for j in range v.N:
                        Uf[:,j] += c[q,j]*d[q,:,j]
                        
                        
                        
                        
This is visualized in the 2D case as summing up product pairs along the third dimension of d and the first dimension c, that is we have the objects (repeated here for convenience):

$$\underline{\underline{c}}_x = \left(\begin{array}{cccccccc}
(-1)^0\beta_0^{,0} & (-1)^0\beta_0^{,1} & \cdots & \cdots & \cdots & (-1)^0\beta_0^{,N_v-2}  & (-1)^0\beta_0^{,N_v-1} \\
(-1)^1\beta_1^{,0} & (-1)^1\beta_1^{,1} & \cdots & \cdots & \cdots & (-1)^1\beta_1^{,N_v-2} & (-1)^1\beta_1^{,N_v-1} \\
(-1)^2\beta_2^{,0} & (-1)^2\beta_2^{,1} & \cdots & \cdots & \cdots & (-1)^2\beta_2^{,N_v-2} & (-1)^2\beta_2^{,N_v-1} \\
\vdots  & \vdots & &  &  & \vdots & \vdots \\
\vdots & \vdots & &  &  & \vdots & \vdots \\
(-1)^{N-2}\beta_{N-2}^{,0} & (-1)^{N-2}\beta_{N-2}^{,1} & \cdots & \cdots & \cdots & (-1)^{N-2}\beta_{N-2}^{,N_v-2} & (-1)^{N-2}\beta_{N-2}^{,N_v -1}\\
(-1)^{N-1}\beta_{N-1}^{,0} & (-1)^{N-1}\beta_{N-1}^{,1} & \cdots & \cdots & \cdots & (-1)^{N-1}\beta_{N-1}^{,N_v-2} & (-1)^{N-1}\beta_{N-1}^{,N_v-1} \\
\end{array}\right)_{N\times N_v}
$$

equivalently,

$$\underline{\underline{c}}_x = \left(\begin{array}{cccccccc}
c_{0,0} & c_{0,1} & \cdots & \cdots & \cdots & c_{0,N_v-2}  & c_{0,N_v-1} \\
c_{1,0} & c_{1,1} & \cdots & \cdots & \cdots & c_{0,N_v-2} & c_{0,N_v-1} \\
\vdots  & \vdots & &  &  & \vdots & \vdots \\
\vdots & \vdots & &  &  & \vdots & \vdots \\
c_{N-2,0} & c_{N-2,0} & \cdots & \cdots & \cdots & c_{N-2,N_v-2} & c_{N-2,N_v-1}\\
c_{N-1,0} & c_{N-1,1} & \cdots & \cdots & \cdots & c_{N-1,N_v-2} & c_{N-1,N_v-1} \\
\end{array}\right)_{N\times N_v}
$$.

where

$$(\underline{\underline{c}})_x = (c_{dn,j}) \qquad \text{for } dn\in [1,2,\ldots N-1],j\in [0,1,\ldots N_v-1]$$ 

Again, each column $j$ has the same coefficients $\{c\}$ for all $x$ at that $v_j$.

and, 

$$d_{i,j}^{(dn)} = \left.(\Delta x)^{dn} \frac{\partial^{dn}f}{\partial x}\right\rvert_{i,j}, \qquad d_{i,j}^{(0)} = f_{i,j}$$

or, in matrix form

$$(\underline{\underline{d}}_x)_{N\times N_x\times N_v} = \left\{\left(\begin{array}{cccccccc}
d^{(0)}_{0,0} & d^{(0)}_{0,1} & \cdots & \cdots & \cdots & d^{(0)}_{0,N_v-2}  & d^{(0)}_{0,N_v-1} \\
d^{(0)}_{1,0} & d^{(0)}_{1,1} & \cdots & \cdots & \cdots & d^{(0)}_{0,N_v-2} & d^{(0)}_{0,N_v-1} \\
\vdots  & \vdots & &  &  & \vdots & \vdots \\
\vdots & \vdots & &  &  & \vdots & \vdots \\
d^{(0)}_{N-2,0} & d^{(0)}_{N-2,0} & \cdots & \cdots & \cdots & d^{(0)}_{N-2,N_v-2} & d^{(0)}_{N-2,N_v-1}\\
d^{(0)}_{N-1,0} & d^{(0)}_{N-1,1} & \cdots & \cdots & \cdots & d^{(0)}_{N-1,N_v-2} & d^{(0)}_{N-1,N_v-1} \\
\end{array}\right)_{N_x\times N_v} \\
\left(\begin{array}{cccccccc}
d^{(1)}_{0,0} & d^{(1)}_{0,1} & \cdots & \cdots & \cdots & d^{(1)}_{0,N_v-2}  & d^{(1)}_{0,N_v-1} \\
d^{(1)}_{1,0} & d^{(1)}_{1,1} & \cdots & \cdots & \cdots & d^{(1)}_{0,N_v-2} & d^{(1)}_{0,N_v-1} \\
\vdots  & \vdots & &  &  & \vdots & \vdots \\
\vdots & \vdots & &  &  & \vdots & \vdots \\
d^{(1)}_{N-2,0} & d^{(1)}_{N-2,0} & \cdots & \cdots & \cdots & d^{(1)}_{N-2,N_v-2} & d^{(1)}_{N-2,N_v-1}\\
d^{(1)}_{N-1,0} & d^{(1)}_{N-1,1} & \cdots & \cdots & \cdots & d^{(1)}_{N-1,N_v-2} & d^{(1)}_{N-1,N_v-1} \\
\end{array}\right)_{N_x\times N_v} \\
\left(\begin{array}{cccccccc}
d^{(2)}_{0,0} & d^{(2)}_{0,1} & \cdots & \cdots & \cdots & d^{(2)}_{0,N_v-2}  & d^{(2)}_{0,N_v-1} \\
d^{(2)}_{1,0} & d^{(2)}_{1,1} & \cdots & \cdots & \cdots & d^{(2)}_{0,N_v-2} & d^{(2)}_{0,N_v-1} \\
\vdots  & \vdots & &  &  & \vdots & \vdots \\
\vdots & \vdots & &  &  & \vdots & \vdots \\
d^{(2)}_{N-2,0} & d^{(2)}_{N-2,0} & \cdots & \cdots & \cdots & d^{(2)}_{N-2,N_v-2} & d^{(2)}_{N-2,N_v-1}\\
d^{(2)}_{N-1,0} & d^{(2)}_{N-1,1} & \cdots & \cdots & \cdots & d^{(2)}_{N-1,N_v-2} & d^{(2)}_{N-1,N_v-1} \\
\end{array}\right)_{N_x\times N_v} \\
\vdots \\
\left(\begin{array}{cccccccc}
d^{(N-1)}_{0,0} & d^{(N-1)}_{0,1} & \cdots & \cdots & \cdots & d^{(N-1)}_{0,N_v-2}  & d^{(N-1)}_{0,N_v-1} \\
d^{(N-1)}_{1,0} & d^{(N-1)}_{1,1} & \cdots & \cdots & \cdots & d^{(N-1)}_{0,N_v-2} & d^{(N-1)}_{0,N_v-1} \\
\vdots  & \vdots & &  &  & \vdots & \vdots \\
\vdots & \vdots & &  &  & \vdots & \vdots \\
d^{(N-1)}_{N-2,0} & d^{(N-1)}_{N-2,0} & \cdots & \cdots & \cdots & d^{(N-1)}_{N-2,N_v-2} & d^{(N-1)}_{N-2,N_v-1}\\
d^{(N-1)}_{N-1,0} & d^{(N-1)}_{N-1,1} & \cdots & \cdots & \cdots & d^{(N-1)}_{N-1,N_v-2} & d^{(N-1)}_{N-1,N_v-1} \\
\end{array}\right)_{N_x\times N_v}\right\}
$$


For example, in the $(i,j)$th entry of Uf[i,j], the prescription $\sum_q c_{q,j}d_{i,j}^q$ as the sum:

$$[Uf]_{i,j} = \sum_q c_{q,j} d_{i,j}^q = c_{0,j} d_{i,j}^0 + c_{1,j} d_{i,j}^1 + c_{2,j} d_{i,j}^2 + \ldots c_{N-1,j} d_{i,j}^{N-1}$$

Viewing the above, this is the same as doing a dot product with the row elements of c and the depth dimension of d at each i,j. The dot product procedure above sums over axis 0 by default and appears the most efficient we can implement this, given that there is not an obvious contraction using routines such as numpy.tensordot for example. <b>This loop will need to be coded in C</b>

### Removing z.MCs (unnecessary calc) in favor of CFL and recasting instance attribute

Actually many changes not cataloged here were made in DECSKS-v2.0, for reasons of time. One key change we wish to bring up is the removal of the unnecessary calculation of moving cell quantities in order to compute CFL numbers per,

    CFL.numbers = (z.MCs - z.prepointmeshvalues) / z.width
    
Since <code>z.MCs = z.prepointmeshvalues + vz.prepointmeshvalues * dt</code>, the <code>z.prepointmeshvalues</code> is subtracted out anyway and this intermediate value of z.MCs is unnecessary. Instead we compute per

    CFL.numbers = vz.prepointmeshvalues * dt / z.width
    
end of story. We capture this as a method internal to the CFL instantiation, which is not housed as a subinstance of each phase space variable being advected z (note we use vz to mean generalized velocity of z, so that it can be a velocity or an acceleration depending on what z is).

A subinstance z.CFL is instantiated per the call in lib.domain as

    self.CFL = CourantNumber(self)

### excerpt of domain.py, CourantNumber() instance

In [ ]:
    def compute_numbers(self, z, vz, dt):
        """Calculates the CFL numbers and corresponding integer and fractional
        parts for each col of z.prepointmesh and stores in the 2D stack

            z.CFL.compute_numbers(z,vz,dt)

        note that each number corresponds to advection in 1D for each 1D problem
        whose subdomain is the column, and whose column space constitutes the entire
        grid.

        Hence, we implement the indicial displacement of each gridpoint according
        to the velocity values in each column by shifting from prepoints

            (i,j) --> (i,j+ CFL.numbers[j])

        where the index i is not explicitly referenced given it is obvious.n       

        inputs:
        self -- (lib.domain.CourantNumber instance) CFL instance with attribute containers
                containers CFL.numbers, CFL.int, CFL.frac.

                NOTE: the method is an attribute belonging to the subinstance z.CFL
                hence, self refers to z.CFL, not z.

        z -- (lib.domain.Setup instance) phasespace instance being advected
        vz -- (lib.domain.Setup instance) velocity for self
        dt -- (float) width of time step, can be a fraction of t.width for split
              methods

        outputs:
        None -- updates attributes
        """
        self.numbers = vz.prepointvaluemesh * dt / z.width

        # if >= 0 , self.int = floor(self.numbers), else ceil(self.numbers)
        # i.e. the sign of CFL.frac agrees with the sign of vz
        self.int = np.where(self.numbers >=0, np.floor(self.numbers),
                            np.ceil(self.numbers))

        # remaining portion is the fractional CFL number
        self.frac = self.numbers - self.int


we set up a unit test of some kind to see the kind of output that is produced here

In [113]:
class phasespace_var:
    def __init__(self, f_old, Nz):
        self.MCs = np.zeros_like(f_old)
        self.prepointvaluemesh = np.random.randn(f_old.shape[0],f_old.shape[1])
        self.prepoints = np.arange(3)
        self.prepointmesh = np.outer( self.prepoints, np.ones([1, 4]) )
        self.width = .5
        self.CFL = CourantNumber(f_old)
        self.postpointmesh = np.zeros_like(f_old)
        self.N = Nz
        

class CourantNumber:
    """Returns a CFL number instance of the phasespace variable z

    inputs:
    z -- (instance) phase space variable from class lib.domain.Setup

    outputs:
    self -- (instance) CFL number ascribed to variable z convection

    Note: broadcasting ensures self.numbers is the same shape as z.MCs
    """
    def __init__(self, f_old):

        self.numbers = np.zeros_like(f_old)
        self.frac = np.zeros_like(f_old)
        self.int = np.zeros_like(f_old)


    def compute_numbers(self, z, vz, dt):
        """Calculates the CFL numbers and corresponding integer and fractional
        parts, call by z.CFL.calculate_numbers(*args)

        inputs:
        self -- (lib.domain.CourantNumber instance) CFL instance with attribute containers
                containers CFL.numbers, CFL.int, CFL.frac.

                NOTE: the method is an attribute belonging to the subinstance z.CFL
                hence, self refers to z.CFL, not z.

        z -- (lib.domain.Setup instance) phasespace instance being advected
        vz -- (lib.domain.Setup instance) velocity for self
        dt -- (float) width of time step, can be a fraction of t.width for split
              methods

        outputs:
        None -- updates attributes
        """        
        self.numbers = vz.prepointvaluemesh * dt / z.width

        # if >= 0 , self.int = floor(self.numbers), else ceil(self.numbers)
        # i.e. the sign of CFL.frac agrees with the sign of vz
        self.int = np.where(self.numbers >=0, np.floor(self.numbers),
                            np.ceil(self.numbers))

        # remaining portion is the fractional CFL number
        self.frac = self.numbers - self.int

        

#### unit test

In [114]:
import numpy as np

f_old = np.random.randn(3,4)
dims = f_old.shape

x = phasespace_var(f_old, 3)
vx = phasespace_var(f_old, 4)

In [115]:
print x.prepointvaluemesh

[[-1.51261694 -1.38824674 -0.11128663 -0.68191783]
 [ 0.30572128 -1.02501623  1.01380546 -2.1141246 ]
 [ 0.8659261  -0.27520949  1.06865567  2.06540774]]


In [116]:
print vx.prepointvaluemesh * dt / x.width

[[ 1.23577391  2.97484115  0.17310995  0.56159815]
 [-5.3228363   4.79991055  1.39992413 -1.69835132]
 [-1.40417029  2.5619856   0.84577965 -2.14487309]]


originally we have an empty container for CFL numbers and so on,

In [117]:
print x.CFL.numbers

[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


To advect the cells, we call the routine z.CFL.compute_numbers(*args) which is an attribute of z.CFL. Choose a time width that is arbitrary,

In [118]:
dt = 1.5

In [119]:
x.CFL.compute_numbers(x, vx, dt)

In [120]:
print x.CFL.numbers

[[ 1.23577391  2.97484115  0.17310995  0.56159815]
 [-5.3228363   4.79991055  1.39992413 -1.69835132]
 [-1.40417029  2.5619856   0.84577965 -2.14487309]]


Verify this computes the correct numbers by hand calculation:

In [121]:
xCFLnumbers = np.zeros_like(f_old) # direct calculation verification

for i in range(vx.prepointvaluemesh.shape[0]):
    for j in range(vx.prepointvaluemesh.shape[1]):
        xCFLnumbers[i,j] = vx.prepointvaluemesh[i,j]*dt / x.width


In [122]:
print xCFLnumbers == x.CFL.numbers

[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]


In [123]:
print xCFLnumbers

[[ 1.23577391  2.97484115  0.17310995  0.56159815]
 [-5.3228363   4.79991055  1.39992413 -1.69835132]
 [-1.40417029  2.5619856   0.84577965 -2.14487309]]


So we can see this produces the expected array.

We also have the corresonding integer and fractional parts:

In [124]:
print x.CFL.int

[[ 1.  2.  0.  0.]
 [-5.  4.  1. -1.]
 [-1.  2.  0. -2.]]


In [125]:
print x.CFL.frac

[[ 0.23577391  0.97484115  0.17310995  0.56159815]
 [-0.3228363   0.79991055  0.39992413 -0.69835132]
 [-0.40417029  0.5619856   0.84577965 -0.14487309]]


In [126]:
print (x.CFL.int + x.CFL.frac) == x.CFL.numbers

[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]


### Discern how to remap all MCs at a given vz[j] all at the same time

We develop a map $\underline{\underline{k}}_{2\times N_x\times N_v}$ whose entries k[0,:,:] and k[1,:,:] contain the collection of contiguous indices $(k_1,k_2)$ that each MC is mapped to originating from that $(i,j)$. This was done with the lib.boundaryconditions.periodic function

### lib.boundaryconditions.periodic

In [127]:
def periodic(sim_params, z, Uf = None):
    """Applies periodic boundary conditions to
    postpoints

    inputs:
    z -- (instance) phase space variable. Used to
         update attribute z.postpointmesh
         (2D mesh of advected gridpoints, not values)

    outputs:
    z.postpointmesh -- (attr) update to z.postpointmesh
                        attribute per periodic BCs
    """

    # there isn't a reason for these if statements
    # should just do the latter, and then for the ballistic
    # push, use k[0,:,:]

    # if calling for Lagrangian integral push
    if Uf is None:
        z.postpointmesh = np.mod(z.postpointmesh, z.N)
        return z.postpointmesh

    # if calling for remap indices


    # TODO this has to be redesigned for 2D, so that the postpoint
    # tensor k.shape = (2, z1.N, z2.N) has the first entry take in lists
    # or manages where the mapping goes in 2D, currently was thinking
    # in only 1D contexts. k1 = [k1x, k1v] --> k2 = [k2x, k2v]

    if Uf is not None:
        # remap index meshes, k1[i,j], k2[i,j] correspond to
        # mapped to indices of prepointmesh[i,j]

        kshape = [2]
        for dim in sim_params['dims']:
            kshape.append(dim)

        k = np.zeros(kshape)
        k[0,:,:] = z.postpointmesh
        k[1,:,:] = np.where(Uf >= 0, np.mod(z.postpointmesh + 1 , z.N),
                      np.mod(z.postpointmesh - 1, z.N))

        return k

### unit test

In [128]:
import numpy as np

Uf = np.random.randn(3,4)
sim_params = {}
sim_params['dims'] = [3,4]

x.postpointmesh = x.prepointmesh + x.CFL.int
x.postpointmesh = periodic(sim_params, x)

In [129]:
k = periodic(sim_params, x, Uf)

In [130]:
print k

[[[ 1.  2.  0.  0.]
  [ 2.  2.  2.  0.]
  [ 1.  1.  2.  0.]]

 [[ 2.  0.  1.  1.]
  [ 1.  1.  1.  2.]
  [ 0.  2.  1.  2.]]]


In [131]:
print k[0,:,:]

[[ 1.  2.  0.  0.]
 [ 2.  2.  2.  0.]
 [ 1.  1.  2.  0.]]


In [134]:
x.prepointvaluemesh[(0,0)]

-1.5126169449982998

In [133]:
print x.prepointvaluemesh

[[-1.51261694 -1.38824674 -0.11128663 -0.68191783]
 [ 0.30572128 -1.02501623  1.01380546 -2.1141246 ]
 [ 0.8659261  -0.27520949  1.06865567  2.06540774]]


In [137]:
print x.prepointmesh

[[ 0.  0.  0.  0.]
 [ 1.  1.  1.  1.]
 [ 2.  2.  2.  2.]]


In [153]:
print x.prepointvaluemesh[ x.prepointmesh.astype(int),vx.prepointmesh.astype(int)]

[[-1.51261694 -1.38824674 -0.11128663 -0.68191783]
 [ 0.30572128 -1.02501623  1.01380546 -2.1141246 ]
 [ 0.8659261  -0.27520949  1.06865567  2.06540774]]


In [142]:
print x.prepointmesh

[[ 0.  0.  0.  0.]
 [ 1.  1.  1.  1.]
 [ 2.  2.  2.  2.]]


x.prepointluemesh[1.,1.]

In [145]:
x.prepointmesh.astype(int)

array([[-1.51261694, -1.38824674, -0.11128663, -0.68191783],
       [ 0.30572128, -1.02501623,  1.01380546, -2.1141246 ],
       [ 0.8659261 , -0.27520949,  1.06865567,  2.06540774]])

In [147]:
x.prepointmesh

array([[ 0.,  0.,  0.,  0.],
       [ 1.,  1.,  1.,  1.],
       [ 2.,  2.,  2.,  2.]])

In [150]:
vx.prepointmesh

array([[ 0.,  0.,  0.,  0.],
       [ 1.,  1.,  1.,  1.],
       [ 2.,  2.,  2.,  2.]])

In [151]:
vx.prepointmesh = np.outer( np.ones([x.N, 1]), np.arange(vx.N))

In [152]:
vx.prepointmesh

array([[ 0.,  1.,  2.,  3.],
       [ 0.,  1.,  2.,  3.],
       [ 0.,  1.,  2.,  3.]])

The map $\underline{\underline{k}}$ gives only the advecting index postpoint, the other postpoint is the same prepoint it had as the nonadvecting variable, e.g. in x advection at a given v[j], i is advected, while j remains the same. We can assemble a 2D mesh of grid indices by referencing indices as:

for x advection:

    f_new[ k[0,:,:], vz.prepointmesh ] <--- f_old
    f_new[ k[1,:,:], vz.prepointmesh ] <--- f_old
    
where the mapping (k[r,:,:], vz.prepointmesh) gives the appropriate map from [i,j] to [i + k[r], j] for r = 0, 1, the contiguous gridpoints that each MC maps to.

In [155]:
f_old

array([[-0.29577547,  1.07971557, -2.51825679,  0.32066116],
       [ 0.24385405, -0.75398576,  0.83250385,  0.5604271 ],
       [-0.43879407,  1.91197068, -0.145558  ,  1.32057128]])

In [156]:
print k[0,:,:]

[[ 1.  2.  0.  0.]
 [ 2.  2.  2.  0.]
 [ 1.  1.  2.  0.]]


Try mapping f_old[:,:] --> f_new[k[0,:,:], vx.prepointmesh]

In [170]:
f_new = np.zeros([3,4])

In [171]:
f_new[ k[0,:,:].astype(int), vx.prepointmesh.astype(int)] += f_old

In [172]:
f_new

array([[ 0.        ,  0.        , -2.51825679,  1.32057128],
       [-0.43879407,  1.91197068,  0.        ,  0.        ],
       [ 0.24385405, -0.75398576, -0.145558  ,  0.        ]])

In [163]:
vx.prepointmesh

array([[ 0.,  1.,  2.,  3.],
       [ 0.,  1.,  2.,  3.],
       [ 0.,  1.,  2.,  3.]])

In [164]:
k[0,:,:]

array([[ 1.,  2.,  0.,  0.],
       [ 2.,  2.,  2.,  0.],
       [ 1.,  1.,  2.,  0.]])

In [169]:
for j in range(4):
    print "j = %d:" % j
    print ""
    for i in range(3):
        print "(%d, %d) -> (%d, %d)" % (i,j, k[0,i,j], vx.prepointmesh[i,j])
    print ""

j = 0:

(0, 0) -> (1, 0)
(1, 0) -> (2, 0)
(2, 0) -> (1, 0)

j = 1:

(0, 1) -> (2, 1)
(1, 1) -> (2, 1)
(2, 1) -> (1, 1)

j = 2:

(0, 2) -> (0, 2)
(1, 2) -> (2, 2)
(2, 2) -> (2, 2)

j = 3:

(0, 3) -> (0, 3)
(1, 3) -> (0, 3)
(2, 3) -> (0, 3)



### smaller example of same thing

In [173]:
K = np.array([[1,0],[1,1]])

In [186]:
F_old = 5*np.random.randn(2,2).astype(int) +  3*np.random.randn(2,2).astype(int)

In [187]:
F_old

array([[  5,  -3],
       [ -3, -10]])

In [188]:
cols = np.outer(np.ones([2,1]), np.arange(2))

In [189]:
cols

array([[ 0.,  1.],
       [ 0.,  1.]])

In [190]:
for j in range(2):
    print "j = %d:" % j
    print ""
    for i in range(2):
        print "(%d, %d) -> (%d, %d)" % (i,j, K[i,j], cols[i,j])
    print ""

j = 0:

(0, 0) -> (1, 0)
(1, 0) -> (1, 0)

j = 1:

(0, 1) -> (0, 1)
(1, 1) -> (1, 1)



In [191]:
K

array([[1, 0],
       [1, 1]])

In [222]:
cols = cols.astype(int)
print cols

[[0 1]
 [0 1]]


In [224]:
F_new = np.zeros([2,2])

In [226]:
F_new[K[0,0],cols[0,0]] += F_old[0,0]

In [227]:
F_new

array([[ 0.,  0.],
       [ 5.,  0.]])

In [228]:
F_new[K[1,0],cols[1,0]] += F_old[1,0]

In [229]:
F_new

array([[ 0.,  0.],
       [ 2.,  0.]])

In [220]:
K[0,0], cols[0,0]

(1, 0.0)

In [212]:
F_new

array([[  0.,  -3.],
       [ -6., -20.]])

In [213]:
F_old

array([[  5,  -3],
       [ -3, -10]])

In [230]:
x.prepoints

array([0, 1, 2])

## Generalizing Beta_matrix(*args) method

#### Current form: x-advection

In [ ]:
def Beta_matrix(sim_params, z, vz):
    """constructs the B matrix, whose columns are
    the beta vectors (shape = N x 1) for each value
    of the generalized velocity for the advecting
    variable z.

    See DECSKS-09 part 1 for details of this calculation

    inputs:
    sim_params -- (dict) simulation parameters
    z.CFL.frac -- (ndarray, ndim=2) contains the fractional CFL numbers
                  for every [i,j]
    """

    # TODO currently thought through for x advection, will need to think about
    # how to make this work for v advection, it is possible all that is needed is a tranpose
    # operation

    # local copies of A matrices
    A_neg = sim_params['A_matrix']['-1']
    A_pos = sim_params['A_matrix']['1']

    N_arr = np.outer(np.arange(1,sim_params['N']+1), np.ones([1,vz.N]))

    # for broadcasting operations below to an N x vz.N array
    # we require an identically dimensioned matrix. Hence, we slice the rows up to N values.
    # There is no loss of information here given that every row entry in the z.CFL.frac matrix
    # is constant, given z.CFL.frac = z.CFL.frac (vz.prepointvaluemesh)

    # Naming per DECSKS-09 notebook:
    #
    #        alpha = z.CFL.frac, shape: (z.N, vz.N)
    #        alpha_hat = truncated z.CFL.frac[:N, :vz.N], vz.N = z.CFL.frac.shape[1]
    #        alpha_tilde : alpha_tilde[q,j] = alpha_hat ** q,q = 0, 1, ... N-1
    #

    alpha_hat = z.CFL.frac[:sim_params['N'],:z.CFL.frac.shape[1]]
    alpha_tilde = ma.array(alpha_hat ** N_arr / scipy.misc.factorial(N_arr))

    mask_neg = (alpha_hat < 0)
    mask_pos = np.logical_not(mask_neg)

    # mask out negative values, leave only positives (>= 0)
    alpha_tilde.mask = mask_neg

    # operate on only positive vlaues (>= 0)
    beta_pos = ma.dot(A_pos, alpha_tilde)

    # mask out positive (>= 0), leave only negatives
    alpha_tilde.mask = mask_pos

    # operate on only negative values
    beta_neg = ma.dot(A_neg, alpha_tilde)

    # consolidate all columns in a single matrix
    B = np.zeros([sim_params['N'], sim_params['N' + z.v_str]])

    # wherever beta_neg.mask is False (i.e. unmasked value), assign beta_neg, otherwise beta_pos
    B = np.where(mask_neg == True, beta_neg.data, beta_pos.data)

    return B


# Extensions to $v$-advection

<b>Recall</b>, for the case of $x$ moving cells in advected, we have $\alpha = \alpha_j$, thus $\beta^{(i,j)}_p = \beta^{,j}_p$ for all $i$ and we save $N_x$ computations by constancy in $i$ at each $j$ (as well as storage). For every $j$ ($v_j$), we can find the associated $N$ correctors $\{\beta^{,j}\}$ that corresponds to all $x_i$ at that velocity as the solution to:

$$\left(\begin{array}{c}
\beta^{,j}_0 \\
\beta^{,j}_1 \\
\beta^{,j}_2 \\
\beta^{,j}_3 \\
\vdots \\
\vdots \\
\beta^{,j}_{N-3} \\
\beta^{,j}_{N-2} \\
\beta^{,j}_{N-1} 
\end{array}\right)_{N\times 1} = \left(\begin{array}{ccc c ccc}
\phantom{\pm}\frac{B_0}{0!} & \phantom{\pm}0 & \cdots & \cdots & \cdots & \cdots & \cdots & \cdots & 0\\
\pm \frac{B_1}{1!} & \phantom{\pm}\frac{B_0}{0!} & 0 & \cdots & \cdots & \cdots & \cdots & \cdots & 0 \\
\phantom{\pm}\frac{B_2}{2!} & \pm \frac{B_1}{1!} & \phantom{\pm}\frac{B_0}{0!} & 0 & \cdots &  \cdots & \cdots &  \cdots & 0 \\
 \phantom{\pm}\frac{B_3}{3!}& \phantom{\pm}\frac{B_2}{2!} & \pm \frac{B_1}{1!} & \frac{B_0}{0!} & 0 &  \cdots & \cdots  &  \cdots &0 \\
\phantom{\pm}\vdots & \phantom{\pm}\vdots & \ddots & \ddots & \ddots & \ddots &  & & 0\\
\phantom{\pm}\vdots & \phantom{\pm}\vdots &  & \ddots & \ddots & \ddots & \ddots & & 0 \\
\phantom{\pm}\frac{B_{N-3}}{(N-3)!} & \phantom{\pm}\frac{B_{N-4}}{(N-4)!} & \cdots & \cdots & \ddots & \pm \frac{B_1}{1!} & \phantom{\pm}\frac{B_0}{0!} & 0 & 0\\
\phantom{\pm}\frac{B_{N-2}}{(N-2)!} & \phantom{\pm}\frac{B_{N-3}}{(N-3)!} & \cdots  & \cdots & \cdots & \phantom{\pm}\frac{B_2}{2!} & \pm \frac{B_1}{1!} & \frac{B_0}{0!} & 0\\
\phantom{\pm}\frac{B_{N-1}}{(N-1)!} & \phantom{\pm}\frac{B_{N-2}}{(N-2)!} & \cdots & \cdots & \cdots & \phantom{\pm}\frac{B_3}{3!} & \phantom{\pm}\frac{B_2}{2!} & \pm \frac{B_1}{1!} & \frac{B_0}{0!}\\
\end{array}\right)_{N\times N} \left(\begin{array}{c}
\frac{\alpha_{,j}^1}{1!} \\
\frac{\alpha_{,j}^2}{2!} \\
\frac{\alpha_{,j}^3}{3!}\\
\vdots \\
\vdots \\
\frac{\alpha_{,j}^{N-2}}{(N-2)!}\\
\frac{\alpha_{,j}^{N-1}}{(N-1)!} \\
\frac{\alpha_{,j}^N}{N!}
\end{array}
\right)_{N\times 1}, \qquad \text{x advection} \qquad \text{for all } j\in [0,1,\ldots , N_v - 1]$$


For $v$-advection, we have $\alpha = \alpha (x_i)$ (correspondingly $\beta = \beta (x_i)$), so the according changeovers required are $\alpha_{,j} \rightarrow \alpha_i$, where the entries are constant for each column entry in a given row (before we had each row being constant for each column). <b>We acknowledge the idea of transposing the CFL matrix for these flux and correction calculations may be advantageous in that we can reuse the above routine, then transpose back</b>. A test of the transpose operation from numpy shows the following sort of time expense

In [3]:
import numpy as np
A = np.random.randn(768, 1536)

In [4]:
%%timeit

np.transpose(A)

The slowest run took 29.84 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 431 ns per loop


So the operational cost, being a tenth of a microsecond is manageable to introduce if desired.

Thus, each $\alpha_i$ is applicable for all $j$ in a row.

$$\left(\begin{array}{c}
\beta^{i}_0 \\
\beta^{i}_1 \\
\beta^{i}_2 \\
\beta^{i}_3 \\
\vdots \\
\vdots \\
\beta^{i}_{N-3} \\
\beta^{i}_{N-2} \\
\beta^{i}_{N-1} 
\end{array}\right)_{N\times 1} = \left(\begin{array}{ccc c ccc}
\phantom{\pm}\frac{B_0}{0!} & \phantom{\pm}0 & \cdots & \cdots & \cdots & \cdots & \cdots & \cdots & 0\\
\pm \frac{B_1}{1!} & \phantom{\pm}\frac{B_0}{0!} & 0 & \cdots & \cdots & \cdots & \cdots & \cdots & 0 \\
\phantom{\pm}\frac{B_2}{2!} & \pm \frac{B_1}{1!} & \phantom{\pm}\frac{B_0}{0!} & 0 & \cdots &  \cdots & \cdots &  \cdots & 0 \\
 \phantom{\pm}\frac{B_3}{3!}& \phantom{\pm}\frac{B_2}{2!} & \pm \frac{B_1}{1!} & \frac{B_0}{0!} & 0 &  \cdots & \cdots  &  \cdots &0 \\
\phantom{\pm}\vdots & \phantom{\pm}\vdots & \ddots & \ddots & \ddots & \ddots &  & & 0\\
\phantom{\pm}\vdots & \phantom{\pm}\vdots &  & \ddots & \ddots & \ddots & \ddots & & 0 \\
\phantom{\pm}\frac{B_{N-3}}{(N-3)!} & \phantom{\pm}\frac{B_{N-4}}{(N-4)!} & \cdots & \cdots & \ddots & \pm \frac{B_1}{1!} & \phantom{\pm}\frac{B_0}{0!} & 0 & 0\\
\phantom{\pm}\frac{B_{N-2}}{(N-2)!} & \phantom{\pm}\frac{B_{N-3}}{(N-3)!} & \cdots  & \cdots & \cdots & \phantom{\pm}\frac{B_2}{2!} & \pm \frac{B_1}{1!} & \frac{B_0}{0!} & 0\\
\phantom{\pm}\frac{B_{N-1}}{(N-1)!} & \phantom{\pm}\frac{B_{N-2}}{(N-2)!} & \cdots & \cdots & \cdots & \phantom{\pm}\frac{B_3}{3!} & \phantom{\pm}\frac{B_2}{2!} & \pm \frac{B_1}{1!} & \frac{B_0}{0!}\\
\end{array}\right)_{N\times N} \left(\begin{array}{c}
\frac{\alpha_{i}^1}{1!} \\
\frac{\alpha_{i}^2}{2!} \\
\frac{\alpha_{i}^3}{3!}\\
\vdots \\
\vdots \\
\frac{\alpha_{i}^{N-2}}{(N-2)!}\\
\frac{\alpha_{i}^{N-1}}{(N-1)!} \\
\frac{\alpha_{i}^N}{N!}
\end{array}
\right)_{N\times 1}, \qquad \text{v advection} \qquad \text{for all } i\in [0,1,\ldots , N_x - 1]$$

Recall the $\alpha$ tensor for $v$-advection takes the form:

$$\underline{\underline{\alpha}} = (\alpha_i) = \left(\begin{array}{ccccc}
\alpha_{0} & \alpha_{0} & \cdots & \alpha_{0} & \alpha_{0} \\
\alpha_{1} & \alpha_{1} & \cdots & \alpha_{1} & \alpha_{1} \\
\vdots & \vdots && \vdots & \vdots \\
\alpha_{N_x-2} & \alpha_{N_x-2} & \cdots & \alpha_{N_x-2} & \alpha_{N_x-2} \\
\alpha_{N_x-1} & \alpha_{N_x-1} & \cdots & \alpha_{N_x-1} & \alpha_{N_x-1} \\
\end{array}
\right)_{N_x\times N_v} \qquad \text{for } v \text{ advection}$$

Or, the transpose ($\alpha_{i,j} \rightarrow \alpha^T_{i,j}$) we recover a similar looking result to the case of $x$-advection:

$$\underline{\underline{\alpha}}^T = (\alpha_{,j}^T) = \left(\begin{array}{ccccc}
\alpha_{,0} & \alpha_{,1} & \cdots & \alpha_{,N_x- 2} & \alpha_{,N_x-1} \\
\alpha_{,0} & \alpha_{,1} & \cdots & \alpha_{,N_x- 2} & \alpha_{,N_x-1} \\
\vdots & \vdots & & \vdots & \vdots \\
\alpha_{,0} & \alpha_{,1} & \cdots & \alpha_{,N_x- 2} & \alpha_{,N_x-1} \\
\alpha_{,0} & \alpha_{,1} & \cdots & \alpha_{,N_x- 2} & \alpha_{,N_x-1} \\
\end{array}
\right)_{N_v \times N_x} \qquad \text{for } v \text{ advection}$$

This reproduces the model used in $x$-advection, so we can then assemble the $\tilde{\alpha}$ tensor as

To compute the scaled vector $\underline{\tilde{\alpha}}_{i} = (\underline{\tilde{\alpha}}_{i})_{N\times 1}$, we use numpy vectorized computations:

    import scipy
    import numpy as np
    
    alpha_tilde_j = CFL.frac[0,j] ** np.arange(1,N+1) / scipy.misc.factorial(np.arange(1,N+1))
    
<b>we can take any i = 0, 1, ..., x.N as every column has constant CFL.frac $\equiv \alpha$</b> (cf. above matrix for $\underline{\underline{\alpha}}$), here arbitrarily take i = 0. In pseudo-math/code, we are computing:

$$(\underline{\tilde{\alpha}_{,j}})_{N\times 1} = \frac{\alpha_j^{(1, 2, \ldots , N)}}{(1, 2, \ldots , N)!}, \qquad j\in [0,1,\ldots , N_v - 1]$$

Which computes by efficient vectorized computation, the following in proper mathematics:

$$\tilde{\alpha}_{,j} = \frac{\alpha_j^p}{p!} \quad \text{where } p\in [1,2,\ldots , N]$$

for some $j$.
    

In [9]:
import numpy as np

sim_params = {}
sim_params['N'] = 21
Nvz = 10

N_arr = np.outer(np.arange(1,sim_params['N']+1), np.ones([1,Nvz]))

print N_arr
print N_arr.shape

print "vs"

N_arr = np.outer(np.ones([1,Nvz]), np.arange(1,sim_params['N']+1))

print N_arr
print N_arr.shape


[[  1.   1.   1.   1.   1.   1.   1.   1.   1.   1.]
 [  2.   2.   2.   2.   2.   2.   2.   2.   2.   2.]
 [  3.   3.   3.   3.   3.   3.   3.   3.   3.   3.]
 [  4.   4.   4.   4.   4.   4.   4.   4.   4.   4.]
 [  5.   5.   5.   5.   5.   5.   5.   5.   5.   5.]
 [  6.   6.   6.   6.   6.   6.   6.   6.   6.   6.]
 [  7.   7.   7.   7.   7.   7.   7.   7.   7.   7.]
 [  8.   8.   8.   8.   8.   8.   8.   8.   8.   8.]
 [  9.   9.   9.   9.   9.   9.   9.   9.   9.   9.]
 [ 10.  10.  10.  10.  10.  10.  10.  10.  10.  10.]
 [ 11.  11.  11.  11.  11.  11.  11.  11.  11.  11.]
 [ 12.  12.  12.  12.  12.  12.  12.  12.  12.  12.]
 [ 13.  13.  13.  13.  13.  13.  13.  13.  13.  13.]
 [ 14.  14.  14.  14.  14.  14.  14.  14.  14.  14.]
 [ 15.  15.  15.  15.  15.  15.  15.  15.  15.  15.]
 [ 16.  16.  16.  16.  16.  16.  16.  16.  16.  16.]
 [ 17.  17.  17.  17.  17.  17.  17.  17.  17.  17.]
 [ 18.  18.  18.  18.  18.  18.  18.  18.  18.  18.]
 [ 19.  19.  19.  19.  19.  19.  19.  19.  19.